In [ ]:
import os
from utils import *
import optuna

In [ ]:
data, usermap, itemmap, users = load_data2()
data_train, data_test, data_val = split_data2(data, 0, 0.2)

In [ ]:
study_name = "IALS"
study = optuna.create_study(
    study_name=study_name,
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

evaluator_validation = EvaluatorHoldout(data_val, cutoff_list=[10])

def objective(trial):
    num_factors = trial.suggest_int('num_factors', 1, 500)
    epochs = trial.suggest_categorical('epochs', [10, 20, 50, 100, 200, 300])
    confidence_scaling = trial.suggest_categorical('confidence_scaling', ["linear", "log"])
    alpha = trial.suggest_float('alpha', 1e-7, 100, log=True)
    epsilon = trial.suggest_float('epsilon', 1e-7, 100, log=True)
    reg = trial.suggest_float('reg', 1e-5, 1, log=True)
    
    recommender = IALSRecommender(data_train, verbose=False)
    recommender.fit(
        epochs=epochs,
        num_factors=num_factors,
        confidence_scaling=confidence_scaling,
        alpha=alpha,
        epsilon=epsilon,
        reg=reg,
        validation_every_n=5,
        stop_on_validation=True,
        evaluator_object=evaluator_validation,
        lower_validations_allowed=5,
        validation_metric="MAP",
    )
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=150)

In [ ]:
study.best_params

In [ ]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

recommender = IALSRecommender(data_train, verbose=False)
recommender.fit(**study.best_params)

In [ ]:
pr, re, ev_map, ev_ndcg, ev_auc = evaluator(recommender, data_train, data_val)
print("Precision: {:.8f}".format(pr))
print("Recall: {:.8f}".format(re))
print("MAP: {:.8f}".format(ev_map))

In [ ]:
submission2(recommender, users, usermap, itemmap, data_train)